# RCOT on SUTs (c>i)

This notebook starts from an exemplary supply-use table (SUT) that features more commodities than industries (c>i). The cases of [c=i](./SU-RCOT_c=i.ipynb) and [c<i](./SU-RCOT_c-i+.ipynb) are illustrated in separate notebooks. All notebooks share the same basic structure. That is, an illustrative SUT is imported from a spreadsheet; solved for the cases of factor constraints being absent or present; and solved when new technologies are introduced.

Since the RCOT models allow for many degrees of freedom, certain aspects are highlighted only for either of the three SUT setups. For example, the issue of monetary or mixed commodity data is illustrated only in the [c=i](./SU-RCOT_c=i.ipynb) notebook. The overlapping parts, however, are explained in detail here and only briefly sketched out for the other two setups. Here, for the most part, we will look at the coefficient formulations of the RCOT models.

<a id='toc'></a>

The outline of the current notebook is as follows:


0. [Data import and set-up](#data-import)
1. [Empirical system in coefficent form w/out factor constraints](#emp_0f)\
    1.1 [As-is solution by inversion (use of constructs)](#emp_0f_inv)\
    1.2 [As-is solution via linear program](#emp_0f_lp)\
    1.3 [Increasing final demand](#emp_0f_lp_e+)\
    1.4 [Reducing final demand](#emp_0f_lp_e-)\
    1.5 [Changing factor use or factor prices](#emp_0f_lp_pi)
2. [Empirical system in coefficient form with factor constraints](#emp_f_lp)\
    2.1 [Increasing final demand](#emp_f_lp_e+)\
    2.2 [Reducing final demand](#emp_f_lp_e-)\
    2.3 [Increasing factor availability](#emp_f_lp_f+)\
    2.4 [Reducing factor availability](#emp_f_lp_f-)
3. [Empirical system in absolute form with factor constraints](#emp_abs)
4. [Adding technology alternatives to the SUT system](#add_jp)\
    4.1 [An alternative for by-product production](#add_jp_bp)\
    4.2 [An alternative for main-product production without secondary products](#add_jp_1mp_sp)\
    4.3 [An alternative for main-product production with secondary products](#add_jp_1mp_jp)\
    4.4 [Multiple alternatives for main-product production](#add_jp_3mp_jp)\
    4.5 [Multiple alternatives for multiple technologies](#add_jp_nmp_sp-jp)    
5. [New alternatives for single production system](#add_sp)

<a id='data-import'></a>

## 0. Data import and set-up

In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c+i-.xlsx")

XLSXFile("../data/SUT_c+i-.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x6           A1:F4        
                  U+e 6x5           A1:E6        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:F4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D6"])
e = convert(Matrix{Float64}, xf["U+e!E2:E6"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

As per the imported spreadsheets, $(V,U,e)$ are given in monetary units meaning that the commodity prices $p=i$. $F$ is in physical units and $\pi$ are the factor prices. As the system is rectangular (c>i), it cannot be solved for total output by inversion. Hence, either we solve via a linear program or we reallocate the by-products via constructs and thus allow for solution by inversion. For convenience, we pack the SUT system into an object that will be used in the following.

In [4]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
# For completeness, let's check the total factor costs (in this unconstrained system)
sut.pii'*sut.f

1×1 Matrix{Float64}:
 169.0

Back to [table of contents](#toc)
<a id='emp_0f'></a>

## 1. Empirical system w/out factor constraint: solution by LP
Since the given SUT-system is rectangular, we cannot perform imputation or impact analyses with it directly by inversion. Instead, we have to resort to either the use of constructs or an optimisation model. Let us first consider the case of no factor-constraints for both. Let us mark the modelled industry output by $g^{*}$ (we will keep this notation also later when we introduce additional technologies).

Back to [table of contents](#toc)
<a id='emp_0f_inv'></a>

### 1.1 Solution by inversion via constructs
Various constructs exist to reallocate by-products. Not all of them work for rectangular SUT-systems, though. For completeness, we introduce the Models A (commodity technology construct, CTC), B (industry technology construct, ITC), C (fixed industry sales structure construct, FISC), and D (fixed product sales structure construct, FPSC) here, although only two of them will be appropriate in the present case of more commodities than industries. We compute the solution here via the ITC and compare it to the commodity output $q$ taken from the SUT.  We do the same for the total factor use $f$.

In [6]:
itc = Constructs.ITC(sut);

In [7]:
# Compute the total output and compare it to the total commodity output; a tolerance measure is required to account for precision deviations
x = itc.L*itc.y
isapprox(x, sut.q, rtol = 1e-10)

true

In [8]:
# The corresponding factor use is calculated and compared to the empirical data; yet again a tolerance measure is required
f = itc.R*itc.L*itc.y
isapprox(f, sut.f, rtol = 1e-10)

true

Back to [table of contents](#toc)
<a id='emp_0f_lp'></a>

### 1.2 Solution by LP
Let us now solve the system via a linear program employing the SUT coefficients.

In [9]:
# To set up our initial primal, we write:
primal = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(primal, g_star[1:size(V,1)] >= 0, base_name = "g*")
@objective(primal, Min, sum(sut.pii'*sut.S*g_star))
@constraint(primal, c1, (sut.C-sut.B)*(g_star) .>= sut.e)

# To print the model:
# print(model) #or:
latex_formulation(primal)

$$ \begin{aligned}
\min\quad & 0.45454545454545453 g*_{1} + 0.6000000000000001 g*_{2} + 0.75 g*_{3}\\
\text{Subject to} \quad & 0.9090909090909091 g*_{1} - 0.24210526315789474 g*_{2} \geq 57\\
 & 0.9263157894736842 g*_{2} - 0.09375 g*_{3} \geq 79\\
 & -0.4318181818181818 g*_{1} - 0.042105263157894736 g*_{2} + 0.71875 g*_{3} \geq 27\\
 & -0.022727272727272728 g*_{1} - 0.042105263157894736 g*_{2} + 0.10416666666666666 g*_{3} \geq 4\\
 & 0.02083333333333333 g*_{3} \geq 2\\
 & g*_{1} \geq 0\\
 & g*_{2} \geq 0\\
 & g*_{3} \geq 0\\
\end{aligned} $$

In [10]:
# Run the model and show results
optimize!(primal)
solution_summary(primal; verbose = true)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.69000e+02
  Objective bound    : -Inf
  Dual objective value : 1.69000e+02
  Primal solution :
    g*[1] : 8.80000e+01
    g*[2] : 9.50000e+01
    g*[3] : 9.60000e+01
  Dual solution :
    c1 : 0.00000e+00

* Work counters
  Solve time (sec)   : 0.00000e+00


We can see in the output that the model found a feasible solution for both the primal and dual, with the resulting objective value being $169$. We arrived at this value earlier when we calculated the total factor costs simply as $\pi' f$. Moreover, the modelled industry output equals the one we calculated directly from the monetary SUT-system earlier.

We can also see, however, that the shadow prices are unequal the unit vector; we should be reminded that $p = i$ because the given system was in monetary values.

In [11]:
@show shadow_price.(c1)
@show isapprox(sut.e, (sut.C - sut.B)*value.(g_star), rtol = 1e-10);

shadow_price.(c1) = [-0.7072094894644295; -1.2093106428635296; -0.0; -8.288379578577178; -0.0;;]
isapprox(sut.e, (sut.C - sut.B) * value.(g_star), rtol = 1.0e-10) = true


Let us now model the dual explicitly and examine this in more detail:

In [12]:
# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@objective(dual, Max, sum(p'*sut.e))
@constraint(dual, c1d, (sut.C-sut.B)'*(p) .<= sut.S'*sut.pii)

# Print the model
latex_formulation(dual)

$$ \begin{aligned}
\max\quad & 57 p_{1} + 79 p_{2} + 27 p_{3} + 4 p_{4} + 2 p_{5}\\
\text{Subject to} \quad & 0.9090909090909091 p_{1} - 0.4318181818181818 p_{3} - 0.022727272727272728 p_{4} \leq 0.45454545454545453\\
 & -0.24210526315789474 p_{1} + 0.9263157894736842 p_{2} - 0.042105263157894736 p_{3} - 0.042105263157894736 p_{4} \leq 0.6000000000000001\\
 & -0.09375 p_{2} + 0.71875 p_{3} + 0.10416666666666666 p_{4} + 0.02083333333333333 p_{5} \leq 0.75\\
 & p_{1} \geq 0\\
 & p_{2} \geq 0\\
 & p_{3} \geq 0\\
 & p_{4} \geq 0\\
 & p_{5} \geq 0\\
\end{aligned} $$

In [13]:
# Run the model and show results
optimize!(dual)
model_solution(primal)
@show value.(p)
@show shadow_price.(c1d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.00000000000006
value.(p) = [1.0561836072199977, 0.9769985726736476, 1.1709128573052583, 0.0, 0.0]
shadow_price.(c1d) = [88.00000000000001; 95.0; 96.00000000000001;;]


While the model has a feasible solution for which the objective value is equal to that one of the primal and where the shadow prices reflect the solved total output of the primal, we see indeed that the modelled price vector is unequal the unit vector (and in fact also unequal the shadow price that we got directly from the primal). Does that mean that the originally implied/assumed price vector $p = i$ is sub-optimal? No, as we can easily see when we calculate the dual's objective function and constraint separately with that vector:

In [14]:
println("The thus calculated objective value is: ", ones(size(p,1))'*sut.e)
println("And the LHS of the constraint: ", (sut.C-sut.B)'*ones(size(p,1)))
println("And the same LHS with new prices: ", (sut.C-sut.B)'*value.(p))

The thus calculated objective value is: [169.0;;]
And the LHS of the constraint: [0.45454545454545453, 0.6, 0.75]
And the same LHS with new prices: [0.4545454545454546, 0.6000000000000001, 0.75]


With that, we see that a system consisting of more commodities than industries may have multiple alternate optimal solutions for its dual. That is, no unique solution for the price vector exists. The usefulness of the dual is hence limited in this case. The reason for this behaviour is, mathematically speaking, the row-wise linear dependence of the net output (coefficients) matrix; it is thus not of full rank. Economically, market actors would realise prices initially in such a way that products with the highest output receive the highest ones while others may be put on the market free of cost; in the present example, assuming that each industry is producing one primary product each next to potentially some secondary products, it is the primary products that receive non-zero prices. As seen above, however, other pricing strategies are possible, too, with an infinite combination of solution vectors being possible.

We will encounter the opposite issue when more industries are present than commodities ([c<i](./SU-RCOT_c-i+.ipynb)), resulting in column-wise linear dependence of the net output matrix. That is, the dual will then reflect the empirical base data whereas the primal yields a non-unique solution.

In the following, we will concentrate on the primal.

Back to [table of contents](#toc)
<a id='emp_0f_inv_e+'></a>

#### 1.3 Increasing final demand
Let's say that we want to increase the final demand for commodity c.5 from 2 to 4:

In [15]:
# Change the respective RHS of the constraint
set_normalized_rhs(c1[5], 4)

In [16]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(g_star)
@show value.(c1)
@show shadow_price.(c1);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 248.00568181818187
value.(g_star) = [90.5875, 104.7159090909091, 192.00000000000003]
value.(c1) = [57.0; 79.0; 94.47357954545457; 13.532102272727274; 4.0;;]
shadow_price.(c1) = [-0.5; -0.778409090909091; -0.0; -0.0; -39.50284090909092;;]


We see that the system generally satisfies the increased final demand (now at a higher cost, cf. objective value), although the total net output is for some commodities (e.g. c.3) considerably higher than what is needed (see below). Such a result is only sensible under the assumption of free disposal.

In [17]:
@show sut.e # The final demand
@show (sut.C-sut.B)*value.(g_star); # The net output w/ computed total industry output

sut.e = [57.0; 79.0; 27.0; 4.0; 2.0;;]
(sut.C - sut.B) * value.(g_star) = [57.0, 79.0, 94.47357954545457, 13.532102272727272, 3.9999999999999996]


Back to [table of contents](#toc)
<a id='emp_0f_lp_e-'></a>

#### 1.4 Reducing final demand
##### 1.4.1 ... for by-product
Let's say that we want to reduce the final demand for the same commodity c.5 down to 0:

In [18]:
# Change the respective RHS of the constraint
set_normalized_rhs(c1[5], 0)

In [19]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(g_star)
@show value.(c1)
@show shadow_price.(c1);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.00000000000003
value.(g_star) = [88.00000000000001, 95.00000000000001, 96.0]
value.(c1) = [57.0; 79.0; 27.0; 3.999999999999999; 1.9999999999999996;;]
shadow_price.(c1) = [-1.0561836072199977; -0.9769985726736479; -1.1709128573052578; -0.0; -0.0;;]


We can see that the solution and objective value are now equal to the results when c.5 = 2. The reason for this is that the demand for primary products is unchanged and must still be satisfied; lowering the demand for a secondary product, however, would imply that the output of associated primary products would have to be reduced. Due to this conflict, the model economy produces still more than needed, at least for c.5. Also here, the assumption of free disposal is implicit.

##### 1.4.2 ... for primary product
Let us now modify the final demand for c.2, which is only produced as a primary product (by i.2).

In [20]:
# Change the respective RHS of the constraint
set_normalized_rhs(c1[5], 2) # c.5 back to the original value
set_normalized_rhs(c1[2], 70) # for c.2 down

In [21]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(g_star)
@show value.(c1)
@show shadow_price.(c1);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 161.9943181818182
value.(g_star) = [85.41250000000001, 85.2840909090909, 96.00000000000001]
value.(c1) = [57.0; 70.0; 28.526420454545462; 4.467897727272727; 2.0;;]
shadow_price.(c1) = [-0.5; -0.778409090909091; -0.0; -0.0; -39.50284090909092;;]


Now we can see that the reduction in final demand for a primary product changes the economic structure indeed. The industries i.1 and i.2 reduce their total output which coincided with a greater commodity suppy of c.3 and c.4 than is demanded.

Back to [table of contents](#toc)
<a id='emp_0f_lp_pi'></a>

### 1.5 Changing factor use or factor prices
In the current model, without a factor constraint, changing factor use or factor prices has no effect on the primal (except for an altered objective value). Such a modification does, however, result in changed commodity prices in the dual. These changes are subject to the above described limitations regarding linear dependence issues, and are therefore not shown here with an example.

Back to [table of contents](#toc)
<a id='emp_f_lp'></a>

## 2. Empirical system w/ factor constraint: solution by LP
Let us set up the model with a factor constraint. We simply choose the observed factor total $f$ for now. Please also note that we now run the optimisation model through a function.

In [22]:
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [23]:
# Run the primal SU-RCOT in its coefficient (relative) form
su_rcot_model = su_rcot("primal", "rel", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.00000000000006
value.(var_con) = [88.00000000000001, 95.00000000000001, 96.00000000000001]
value.(demand_con) = [57.0, 79.0, 27.000000000000007, 4.0, 1.9999999999999998]
value.(factor_con) = [34.16666666666667, 16.111111111111114, 2.2222222222222223, 30.476190476190478]


With the factor constraint chosen as of now, the model reproduces unsurprisingly the observed data.

Back to [table of contents](#toc)
<a id='emp_f_lp_e+'></a>

### 2.1 Increasing final demand

Let us now increase the final demand for #c.5 as before. Before doing so, let us make the model constraint easily accessible.

In [24]:
# Overview of constraint types in model
list_of_constraint_types(su_rcot_model)

3-element Vector{Tuple{Type, Type}}:
 (AffExpr, MathOptInterface.GreaterThan{Float64})
 (AffExpr, MathOptInterface.LessThan{Float64})
 (VariableRef, MathOptInterface.GreaterThan{Float64})

In [25]:
# Access all constraints of the model, including the variables
model_con = all_constraints(su_rcot_model; include_variable_in_set_constraints = true)

12-element Vector{ConstraintRef}:
 c1 : 0.9090909090909091 g*[1] - 0.24210526315789474 g*[2] >= 57
 c1 : 0.9263157894736842 g*[2] - 0.09375 g*[3] >= 79
 c1 : -0.4318181818181818 g*[1] - 0.042105263157894736 g*[2] + 0.71875 g*[3] >= 27
 c1 : -0.022727272727272728 g*[1] - 0.042105263157894736 g*[2] + 0.10416666666666666 g*[3] >= 4
 c1 : 0.02083333333333333 g*[3] >= 2
 c2 : 0.19886363636363638 g*[1] + 0.1754385964912281 g*[2] <= 34.16666666666667
 c2 : 0.01893939393939394 g*[1] + 0.07309941520467836 g*[2] + 0.078125 g*[3] <= 16.11111111111111
 c2 : 0.007797270955165692 g*[2] + 0.015432098765432098 g*[3] <= 2.2222222222222223
 c2 : 0.07034632034632034 g*[1] + 0.05012531328320802 g*[2] + 0.20337301587301584 g*[3] <= 30.476190476190474
 g*[1] >= 0
 g*[2] >= 0
 g*[3] >= 0

In [26]:
# We are interested in the supply-demand constraints, therefore:
demand_con = all_constraints(su_rcot_model, AffExpr, MOI.GreaterThan{Float64})

5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 c1 : 0.9090909090909091 g*[1] - 0.24210526315789474 g*[2] >= 57
 c1 : 0.9263157894736842 g*[2] - 0.09375 g*[3] >= 79
 c1 : -0.4318181818181818 g*[1] - 0.042105263157894736 g*[2] + 0.71875 g*[3] >= 27
 c1 : -0.022727272727272728 g*[1] - 0.042105263157894736 g*[2] + 0.10416666666666666 g*[3] >= 4
 c1 : 0.02083333333333333 g*[3] >= 2

In [27]:
# The factor constraints are the following:
factor_con = all_constraints(su_rcot_model, AffExpr, MOI.LessThan{Float64})

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c2 : 0.19886363636363638 g*[1] + 0.1754385964912281 g*[2] <= 34.16666666666667
 c2 : 0.01893939393939394 g*[1] + 0.07309941520467836 g*[2] + 0.078125 g*[3] <= 16.11111111111111
 c2 : 0.007797270955165692 g*[2] + 0.015432098765432098 g*[3] <= 2.2222222222222223
 c2 : 0.07034632034632034 g*[1] + 0.05012531328320802 g*[2] + 0.20337301587301584 g*[3] <= 30.476190476190474

In [28]:
# And the variables are these:
var_con = all_constraints(su_rcot_model, VariableRef, MOI.GreaterThan{Float64})

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 g*[1] >= 0
 g*[2] >= 0
 g*[3] >= 0

In [29]:
# Modify the chosen constraint as described above:
set_normalized_rhs(demand_con[5], 4)

In [30]:
# Run the model and show results
optimize!(su_rcot_model)
@show termination_status(su_rcot_model)
@show primal_status(su_rcot_model)
@show dual_status(su_rcot_model);

termination_status(su_rcot_model) = MathOptInterface.INFEASIBLE
primal_status(su_rcot_model) = MathOptInterface.NO_SOLUTION
dual_status(su_rcot_model) = MathOptInterface.INFEASIBILITY_CERTIFICATE


We can see that the problem becomes infeasible, because all available factors were already in use and no reallocation of production to otherwise available (cheaper) sectors could take place.

Back to [table of contents](#toc)
<a id='emp_f_lp_e-'></a>

### 2.2 Decreasing final demand
#### 2.2.1 ... for by-product

If we were to reduce the final demand for that respective commodity, however, the problem stays feasible and the solution is as follows:

In [31]:
# Change the respective RHS of the constraint
set_normalized_rhs(demand_con[5], 0)

In [32]:
# Run the model and show results
optimize!(su_rcot_model)
model_solution(su_rcot_model)
show_primal_lhs(su_rcot_model);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.00000000000006
value.(var_con) = [88.00000000000003, 95.00000000000001, 96.00000000000003]
value.(demand_con) = [57.0, 79.0, 27.0, 3.9999999999999996, 2.0000000000000004]
value.(factor_con) = [34.16666666666668, 16.111111111111114, 2.2222222222222228, 30.476190476190474]


The industry output remains unchanged, thus yet again invoking the free-disposal assumption, as the net output of #c.5=2 is greater than the final demand for it (=0).

Back to [table of contents](#toc)

#### 2.2.2 ... for primary product

When we reduce, however, the final demand #c.2, which is only produced as a primary product (by #i.2), we observe the following changes:

In [33]:
# Change the respective RHS of the constraint
set_normalized_rhs(demand_con[5], 2) # c.5 back to the original value
set_normalized_rhs(demand_con[2], 70) # for c.2 down

In [34]:
# Run the model and show results
optimize!(su_rcot_model)
model_solution(su_rcot_model)
show_primal_lhs(su_rcot_model);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 161.99431818181822
value.(var_con) = [85.41250000000002, 85.2840909090909, 96.00000000000001]
value.(demand_con) = [57.0, 70.0, 28.526420454545462, 4.467897727272728, 2.0]
value.(factor_con) = [31.947561553030308, 15.351878156565657, 2.1464646464646466, 29.807156385281388]


These results are equivalent to those when no factor constraints were present (section 1.4.2).

Back to [table of contents](#toc)
<a id='emp_f_lp_f+'></a>

### 2.3 Increase factor availability
All else equal, when we increase any/ all of the available factors, nothing changes in the output structure (or in the model solution overall), because the model does not expand beyond what is required to satisfy the given final demand.

In [35]:
# Change the respective RHS of the constraints
set_normalized_rhs(demand_con[2], 79) # c.2 back to the original value
set_normalized_rhs(factor_con[2], 50) # for f.2 up

In [36]:
# Run the model and show results
optimize!(su_rcot_model)
model_solution(su_rcot_model)
show_primal_lhs(su_rcot_model);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.00000000000003
value.(var_con) = [88.00000000000003, 95.00000000000001, 96.0]
value.(demand_con) = [57.0, 79.0, 26.999999999999986, 3.999999999999998, 2.0]
value.(factor_con) = [34.16666666666668, 16.111111111111107, 2.2222222222222223, 30.47619047619047]


Back to [table of contents](#toc)
<a id='emp_f_lp_f-'></a>

### 2.4 Decrease factor availability
If we were to reduce any of the available factor endowments, however, the problem becomes infeasible, as seen here for reducing the availability of #f.2 down to 14:

In [37]:
# Change the respective RHS of the constraints
set_normalized_rhs(factor_con[2], 14) # for f.2 down

In [38]:
# Run the model and show results
optimize!(su_rcot_model)
model_solution(su_rcot_model)
show_primal_lhs(su_rcot_model);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 169.00000000000003
value.(var_con) = [88.00000000000003, 95.00000000000001, 96.0]
value.(demand_con) = [57.0, 79.0, 26.999999999999986, 3.999999999999998, 2.0]
value.(factor_con) = [34.16666666666668, 16.111111111111114, 2.2222222222222223, 30.47619047619047]


In [39]:
# Let's see if the same holds if all factor availabilities are reduced:
for i in 1:length(su_rcot_data.ϕ)
    set_normalized_rhs(factor_con[i], 0.8.*su_rcot_data.ϕ[i])
end

optimize!(su_rcot_model)
model_solution(su_rcot_model);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 169.00000000000006


The reason for this is that the modelled economy simply cannot satisfy the induced final demand with a lower physical resource base, regardless of the chosen factor prices.

Back to [table of contents](#toc)
<a id='emp_abs'></a>

## 3. A formulation in absolute form
Until now, we have worked with the SUT system in coefficient form. Let us now do the same in absolute form. Note that we assume our factor constraints again to be $phi = f$. We start with the primal and see that the activity levels are as expected unitary.

In [40]:
# Instantiate and run the model and show the solution
su_rcot_primal_abs = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(var_con) = [1.0, 1.0, 1.0]
value.(demand_con) = [57.0, 79.0, 26.999999999999996, 4.0, 2.0]
value.(factor_con) = [34.16666666666667, 16.11111111111111, 2.2222222222222223, 30.476190476190474]


Let us now for completeness also model the dual in absolute form. (We see again that the prices are not unitary.)

In [41]:
# Instantiate and run the model and show the solution
su_rcot_dual_abs = su_rcot("dual", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.0561836072199977, 0.9769985726736475, 1.1709128573052583, 0.0, 0.0]
value.(r) = [0.0, 0.0, 0.0, 0.0]
value.(profit_con) = [40.0, 57.0, 72.0]


Back to [table of contents](#toc)
<a id='add_jp'></a>

## 4. Adding technology alternatives
So far, we have only worked with the SUT system as imported. Let us now add new technologies to it and see how substitution may or may not occur. We use the model in its absolute form.

Back to [table of contents](#toc)
<a id='add_jp_bp'></a>

### 4.1 An alternative for by-product production

First with no changed variables except the added technology.

In [42]:
# Define supply and use of the additional technology
V3_alt1 = [0 0 0 0 10]
U3_alt1 = [0 3 1 0 0]'
F3_alt1 = [1.2 3.6 0 4.2]';

In [43]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V3_alt1]
su_rcot_data.U = @views [su_rcot_data.U U3_alt1]
su_rcot_data.F = @views [su_rcot_data.F F3_alt1];

In [44]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(var_con) = [1.0, 1.0, 1.0, 0.0]
value.(demand_con) = [57.0, 79.0, 26.999999999999996, 4.0, 2.0]
value.(factor_con) = [34.16666666666667, 16.11111111111111, 2.2222222222222223, 30.476190476190474]


We can see that the activity level of technology #t.4 is zero and that the objective value remains unchanged. Hence, no substitution occurs between the alternatives, nor any other changes. 

Let us now assume that the factor requirements of all technologies are lower than originally (by 20%) while the factor endowment stays as before, and that the final demand for #c.5 increases (from 2 to 3.5).

In [45]:
su_rcot_data.F = su_rcot_data.F.*0.8
su_rcot_data.e[5] = 3.5;

In [46]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 138.46403133681972
value.(var_con) = [1.001561810996558, 1.0054323860749839, 1.0033392826903056, 0.1493321434619389]
value.(demand_con) = [57.0, 79.0, 27.0, 4.008539660610004, 3.5]
value.(factor_con) = [27.57098935934173, 13.371263495501985, 1.7849546378996117, 24.96329428546359]


With the adopted changes, we can see that now substitution occurs as the activity level of technology #t.4 is greater than zero. This substitution allows to reduce the factor costs of the system. It shall be noted that a small surplus of commodity #c.4 results.

Back to [table of contents](#toc)
<a id='add_jp_1mp_sp'></a>

### 4.2 An alternative for main-product production without secondary products

Let us now examine the competitiveness of an alternative technology for industry #i.2, which is a single production technology, producing only #c.2. We keep the factor requirements as before but set the final demand for #c.5 back to its original value.

In [47]:
su_rcot_data.V[4,:] = [0 95 0 0 0]
su_rcot_data.U[:,4] = [21 7 6 1 0]'
su_rcot_data.F[:,4] = [14 5 0 4]'
su_rcot_data.e[5] = 2;

In [48]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 132.98003365256886
value.(var_con) = [0.9754184431258935, 0.0, 1.0155864529433347, 1.0015940690510228]
value.(demand_con) = [57.0, 79.0, 27.0, 7.203433574130537, 2.0311729058866694]
value.(factor_con) = [27.67817517047683, 12.40204698708298, 1.2036580183032115, 24.699513164323456]


We see that the model is feasible and that (given the constraints set here) no complementation for #i.2 occurs. The new alternative technology is chosen but at the cost of a commodity surplus. To suppress that surplus, let us convert the supply-demand inequality into an equality and see which technology is then chosen.

In [49]:
su_rcot("primal", "abs", su_rcot_data; surplus=false);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 135.20000000000002
value.(var_con) = [1.0000000000000002, 1.0000000000000002, 1.0, -1.8707674576077474e-16]
value.(demand_con_nosurp) = [57.0, 79.0, 27.0, 4.0, 2.0]
value.(factor_con) = [27.333333333333343, 12.888888888888891, 1.7777777777777777, 24.38095238095238]


We see that the system defaults back to the original setup of dominant technologies, because the earlier surplus cannot be taken up by any technology. (Note that the factor costs (objective value) are higher. This is due to the assumption of free disposal of the surplus.)

Back to [table of contents](#toc)
<a id='add_jp_1mp_jp'></a>

### 4.3 An alternative for main-product production with secondary products

Let us assume that industry #i.3's main product is commodity #c.3. Let the alternative technology now be a single production process, while the dominant technology is a joint production process producing #c.3, #c.4, and #c.5. Let us further assume that we have again an increased final demand for #c.5.

In [50]:
su_rcot_data.V[4,:] = [0 0 20 0 0]
su_rcot_data.U[:,4] = [0 2 5 0 0]'
su_rcot_data.F[:,4] = [0 9.6 0 13.6]'
su_rcot_data.e[5] = 3.5;

In [51]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 182.60340909090917
value.(var_con) = [1.0220525568181826, 1.0767045454545456, 1.7500000000000009, 0.0]
value.(demand_con) = [57.0, 79.0, 77.60518465909092, 11.149076704545458, 3.5]
value.(factor_con) = [28.664796401515165, 17.84442866161617, 2.7121212121212124, 36.49665854978356]


We see that the problem has become infeasible. The alternative technology requires too high factor inputs and the existing system cannot satisfy the final demand.

Let the alternative now be a joint production process.

In [52]:
su_rcot_data.V[4,:] = [0 2 20 0 6]
su_rcot_data.U[:,4] = [0 1 4 5 0]'
su_rcot_data.F[:,4] = [0.3 11 0 6]';

In [53]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 153.26136437548695
value.(var_con) = [1.0025566160777803, 1.008892577661845, 1.1106300179545379, 0.21312332734848743]
value.(demand_con) = [57.0, 79.0, 37.91072275483587, 4.0, 3.5]
value.(factor_con) = [27.551630658784738, 15.94983762811899, 1.9141645117457307, 27.43416568666831]


Now we see that substitution occurs as technology #t.4's activity level is greater than zero. As compared to before, the total factor costs are lower now, but a commodity surplus persists.

Back to [table of contents](#toc)
<a id='add_jp_3mp_jp'></a>

### 4.4 Multiple alternatives for main-product production

Let us now add yet two more alternatives for industry #i.3. The factor requirements as well as the increased final demand for #c.5 are kept as before. Let us moreover reduce the availability of factor #f.1 from 34.2 to 27.8.

In [54]:
# Add the additional parameters to the copied SUT struct
V3_alt = [0 1 17 1 2; 0 0 15 0 4]
U3_alt = [0 8 2 0 1; 1 9 4 0 0]'
F3_alt = [0 1 1 6; 0 2 0 3]'

su_rcot_data.V = @views [su_rcot_data.V; V3_alt]
su_rcot_data.U = @views [su_rcot_data.U U3_alt]
su_rcot_data.F = @views [su_rcot_data.F F3_alt]


su_rcot_data.ϕ[1] = 27.8;

In [55]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 148.68280487943082
value.(var_con) = [1.0088186690752972, 1.0232000185219157, 1.0674966067576475, 0.1129057310676433, 0.0, 0.17189310001971128]
value.(demand_con) = [57.0, 79.0, 34.926672164627824, 4.0, 3.5]
value.(factor_con) = [27.8, 15.020264988440424, 1.8715219152813098, 26.760353214183688]


We can see now that the system's production (in particular industry #i.3) is complemented by technologies #t.4 and #t.6. The factor constraint for #f.1 is active and a small commodity surplus for #c.3 is present. With the given parameterisation, converting the supply-demand inequality into an equality would render the system infeasible:

In [56]:
su_rcot("primal", "abs", su_rcot_data, surplus=false);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 136.520564657843
value.(var_con) = [1.025452851267691, 1.061205538128296, 0.9529193234428893, -0.15330692412393446, 0.0, 0.6285007244644569]
value.(demand_con_nosurp) = [57.0, 79.0, 27.0, 4.0, 3.5]
value.(factor_con) = [28.459755015554446, 12.550998015514871, 1.7582484059342665, 24.97047385109601]


Back to [table of contents](#toc)
<a id='add_jp_nmp_sp-jp'></a>

### 4.5 Multiple alternatives for multiple technologies

We have until now only considered technologies that have their primary output in common with only one dominant technology. Let us now introduce multiple alternatives for multiple industries. These new alternatives may produce in addition to a primary product also a secondary product. We continue from the parameterisation of the previous example.

In [57]:
# Alternatives for #i.2
V2_alt = [0 95 0 0 0; 0 95 0 0 0; 10 60 0 0 0]
U2_alt = [21 7 4 1 0; 14 0 3 1 0; 18 2 16 0 0]'
F2_alt = [13 6 0.6 3; 12 2 7 0; 9 1 0 5]'
# Alternatives for #i.3
V3x_alt = [0 0 0 0 10]
U3x_alt = [0 3 1 0 0]'
F3x_alt = [1.2 3.6 0 4.2]'
# Add Alternatives
su_rcot_data.V = @views [su_rcot_data.V[1:2, :]; V2_alt; su_rcot_data.V[3:end, :]; V3x_alt]
su_rcot_data.U = @views [su_rcot_data.U[:, 1:2] U2_alt su_rcot_data.U[:, 3:end] U3x_alt]
su_rcot_data.F = @views [su_rcot_data.F[:, 1:2] F2_alt su_rcot_data.F[:, 3:end] F3x_alt];

In [58]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 135.88524011731593
value.(var_con) = [0.9681764496400506, 0.0, 0.9263137410518911, 0.07153767207959473, 0.0, 0.9835342928723738, 0.0, 0.0, 0.0, 0.1532931414255253]
value.(demand_con) = [57.0, 79.0, 27.0, 6.901138616312153, 3.5]
value.(factor_con) = [26.63895276330106, 13.444920789690071, 2.2222222222222223, 23.579419980224017]


Compare this result to when we prevent a commodity surplus by converting the supply-demand constraint into an inequality:

In [59]:
su_rcot("primal", "abs", su_rcot_data; surplus=false);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 137.83260140853824
value.(var_con) = [1.0002004052778635, 0.7760595503236308, 0.17438970101302786, 0.07838211320108414, 0.0, 0.9357410826064362, 0.0, 0.0, 0.40712945869678197, 0.0]
value.(demand_con_nosurp) = [57.0, 79.0, 27.0, 4.0, 3.5]
value.(factor_con) = [27.557917816454207, 13.276850332125395, 2.2222222222222223, 24.269732777686542]


We see that a different set of technologies is activated depending on if a commodity surplus is permitted or not (#t.(1, 3, 4, 6, 10) vs #t.(1, 2, 3, 4, 6, 9)).

Lastly, it must be noted that the resolution of the data plays a central role. In all above examples, a small economy was modelled. The effect of connection between technologies via their secondary outputs is non-negligible.

Back to [table of contents](#toc)
<a id='add_sp'></a>

## 5. New alternatives for single production
In contrast to the two RCOT models in coefficient and absolute form above, the model by Duchin & Levine (2011) was formulated in coefficent form for single production. That is, by-products are reallocated. While this can be consistently done via any construct for the base SUT, it remains a challenge to keep this consistency when adding new technologies. To illustrate the challenge of adding new technologies based on observed supply-use information, we start from the underlying SUT, apply a construct of choice, try to reallocate the by-products of the alternative technologies (and adjust all other system variables accordingly), and then solve the IO-RCOT model.

This is easier said than done because the reallocation of by-products can be tricky for new technologies producing multiple commodities. Moreover, the data for the additional technologies must be converted from absolute to coefficient form.

One can argue that, if provided that:
- the primary output of both the dominant and new alternative technology is given in the same unit
- AND the new technology produces only a single output
- AND no additional data is given to allow for a more sophisticated conversion,

the most straightforward way of deriving single production coefficients for such an alternative is to divide the respective matrices by the total output. This strategy is equivalent to the idea behind the lump-sum construct.

### Single production alternative

Let us now work with a single production system based on the ITC. To do so (and to keep the final IO-RCOT model sort of comparable with SU-RCOT), we do:

In [60]:
# Copy the underlying SUT
sut_io = deepcopy(sut)
# Let us adjust the factor requirements as we did earlier for SU-RCOT, now simply for factor intensities
sut_io.S = sut_io.S.*0.8
# Set up the ITC model
itc = Constructs.ITC(sut_io)
# Put the data into a to-be-augmented structure
io_rcot_data = Model_data.IO(itc);

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


Let us now introduce an alternative for producing #c.5 (with data as used earlier with SU-RCOT in [section 4.1](#add_jp_bp)):

In [61]:
# Define supply and use of the additional technology
V3_alt1 = [0 0 0 0 10]
U3_alt1 = [0 3 1 0 0]'
F3_alt1 = [1.2 3.6 0 4.2]';

Since the new technology here only produces one output and because the commodity supply-use data is given in monetary units, we can easily work with that to derive the following coefficients which we then add to our data structure:

In [62]:
# Calculate the coefficients based on total output
x5_alt1 = sum(V3_alt1)
I5_alt1 = V3_alt1'/x5_alt1
A5_alt1 = U3_alt1/x5_alt1
R5_alt1 = F3_alt1/x5_alt1;

In [63]:
# Add the additional parameters to the copied SUT struct
io_rcot_data.I_mod = @views [io_rcot_data.I_mod I5_alt1]
io_rcot_data.A = @views [io_rcot_data.A A5_alt1]
io_rcot_data.R = @views [io_rcot_data.R R5_alt1];

In [64]:
io_rcot("primal", "rel", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 135.2
value.(var_con) = [88.0, 95.0, 76.00000000000001, 10.0, 10.0, 0.0]
value.(demand_con) = [57.0, 79.0, 27.0, 4.0, 2.0]
value.(factor_con) = [27.333333333333336, 12.88888888888889, 1.7777777777777781, 24.38095238095238]


Let us now assume, again as in [section 4.1](#add_jp_bp), that we have an increased demand for commodity #c.5 = 3.5:

In [65]:
io_rcot_data.y[5] = 3.5;
io_rcot("primal", "rel", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 136.4
value.(var_con) = [88.04912621359223, 95.18446601941747, 76.16188241639699, 10.008883495145632, 11.651887810140236, 0.0]
value.(demand_con) = [57.0, 79.0, 27.0, 4.0, 3.5]
value.(factor_con) = [27.367038834951458, 13.014336569579287, 1.8014303408046666, 24.687657060666766]


As compared to SU-RCOT, where we could observe a complementation of dominant and new technology, we see here that the new technology is not employed but simply output levels of all dominant technologies raised. Such substitution/complementation occurs only once a factor constraint is reached, e.g. by driving the demand for the bespoke commodity higher:

In [66]:
io_rcot_data.y[5] = 30.5
io_rcot("primal", "rel", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 158.8204310344827
value.(var_con) = [88.95965517241379, 98.60344827586206, 79.10290229885058, 10.173534482758619, 40.90287356321842, 0.4454022988505486]
value.(demand_con) = [57.0, 79.0, 27.0, 4.0, 30.5]
value.(factor_con) = [28.04520114942529, 15.410708812260527, 2.2222222222222223, 30.327446633825932]


(Interestingly, such a high level of final demand for #c.5 would not be feasible in SU-RCOT anymore.)

### Joint production alternative (monetary)

In the case of the new technology producing multiple outputs AND as long as the supply-use data for it is given in monetary units, one may still operate along the lines of the Lump-Sum construct - despite its conceptual limitations. That is, to now add up the outputs of the new technology and based on that derive coefficient vectors which are then appended to the data structure.

(Another straightforward alternative would be to convert the new technology data along the lines of the by-product technology construct (or Stone method); as compared to the lump-sum approach, the by-product technology construct route would require a choice of which output is the primary output of the new technology.)

As in [section 4.3](#add_jp_1mp_jp), let us add an alternative for producing commodity #c.3:

In [67]:
# Define supply and use of the additional technology
V3_alt1 = [0 2 20 0 6]
U3_alt1 = [0 1 4 5 0]'
F3_alt1 = [0.3 11 0 6]';

In [68]:
# Calculate the coefficients based on total output
x3_alt1 = sum(V3_alt1)
I3_alt1 = io_rcot_data.I_mod[:,3]
A3_alt1 = U3_alt1/x3_alt1
R3_alt1 = F3_alt1/x3_alt1;

In [69]:
# Add the additional parameters to the data structure
io_rcot_data.I_mod[:,6] = I3_alt1
io_rcot_data.A[:,6] = A3_alt1
io_rcot_data.R[:,6] = R3_alt1
# We also set the final demand for #c.5 back to
io_rcot_data.y[5] = 3.5;

In [70]:
io_rcot("primal", "rel", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 136.4
value.(var_con) = [88.04912621359223, 95.18446601941747, 76.16188241639699, 10.008883495145632, 11.651887810140236, 0.0]
value.(demand_con) = [57.0, 79.0, 27.0, 4.0, 3.5]
value.(factor_con) = [27.367038834951458, 13.014336569579287, 1.8014303408046666, 24.687657060666766]


Also here, without a factor constraint being active, no substitution or complementation occurs.

### Joint production alternative (mixed units)

If the new technology produces however multiple outputs for which the supply-use data is given in mixed units, then no such straightforward work-around is possible. While this issue is part of a central argument in our paper, it shall not be our focus now to find (and compare) ways to derive a meaningful IO-RCOT parametrisation for this case, and instead leave that to further research.

Back to [table of contents](#toc)

---

We end here the exposition of RCOT models for the case (c>i) and suggest the reader to continue with case [(c<i)](./SU-RCOT_c-i+.ipynb).